In [160]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import datetime,os

import torch
import pandas as pd

import pickle

from fuzzywuzzy import fuzz

import re
from itertools import product

In [161]:
newdatapath = '..\\..\\..\\..\\TF_data\\ADMIN\\v2\\'
storepath = '..\\..\\..\\..\\TF_data\\ADMIN\\v2\\04_DATA\\'

In [242]:
col_rename_dict = {
    'Valutadatum':'Date_ordered', 
    'Buchungstag':'Date_booked', 
    'Buchungstext':'Text_transaction', 
    'Verwendungszweck':'Use',
    'Beguenstigter/Zahlungspflichtiger':'Contact', 
    'Kontonummer/IBAN':'AccNum',
    'Kontonummer':'AccNum',
    'Betrag':'Value_transaction',
    'Auftragskonto':'OrderAccount', 
    'Glaeubiger ID':'LenderID', 
    'Mandatsreferenz':'MandateReference',
    'Kundenreferenz (End-to-End)':'CustomerReference', 
    'Sammlerreferenz':'CollectorReference',
    'Lastschrift Ursprungsbetrag':'Amount0', 
    'Auslagenersatz Ruecklastschrift':'Amount1',
    'BIC (SWIFT-Code)':'BIC', 
    'BLZ':'BIC',
    'Waehrung':'Currency', 
    'Info':'Information'
}

In [286]:
df_list = []

c_path = newdatapath+'01_SingleTransactions\\'
for c_file in os.listdir(c_path):
    f_name = c_file.split('.')[0].lower()
    f_term = c_file.split('.')[-1].lower()
    if f_term=='xls' or f_term=='xlsx':
        df_list.append(pd.read_excel(c_path+c_file).drop(columns=[
            'Glaeubiger ID',
            'Mandatsreferenz',
            'Kundenreferenz (End-to-End)',
            'Sammlerreferenz',
            'Lastschrift Ursprungsbetrag',
            'Auslagenersatz Ruecklastschrift'
        ],errors='ignore').rename(columns=col_rename_dict))
        if c_file=='20191215.xlsx':
            df_list[-1]['Value_transaction'] = df_list[-1]['Value_transaction'].apply(lambda x: float(str(x)[:-2]+'.'+str(x)[-2:]))
    elif f_term=='csv':
        df_list.append(pd.read_csv(c_path+c_file,sep=';',encoding='latin-1').drop(columns=[
            'Glaeubiger ID',
            'Mandatsreferenz',
            'Kundenreferenz (End-to-End)',
            'Sammlerreferenz',
            'Lastschrift Ursprungsbetrag',
            'Auslagenersatz Ruecklastschrift'
        ],errors='ignore').rename(columns=col_rename_dict))
        if c_file=='20191215.xlsx':
            df_list[-1]['Value_transaction'] = df_list[-1]['Value_transaction'].apply(lambda x: float(str(x)[:-2]+'.'+str(x)[-2:]))
    else:
        #print('Skipping file {}'.format(c_file))
        continue
    #print('Loaded file {}'.format(c_file))

In [287]:
df = pd.concat(df_list).reset_index(drop=True)

In [288]:
df['Use'] = df['Use'].apply(lambda x: 'Miete fuer Haesschen,du bist toll .)' if 'miete fuer haesschen' in x.lower() else x)
df['Use'] = df['Use'].apply(lambda x: 'lohn/gehalt' if 'lohn/gehalt' in x.lower() else x)

In [289]:
df['Date_booked'] = pd.to_datetime(df['Date_booked'],format='%d.%m.%y')
df['Date_ordered'] = pd.to_datetime(df['Date_ordered'],format='%d.%m.%y')

In [290]:
df['Value_transaction'] = df['Value_transaction'].apply(lambda x: str(x).replace(',','.')).astype(np.float64)

In [291]:
for colname in ['Text_transaction','Use','Contact']:
    df[colname] = df[colname].map(lambda x: re.sub(r'\W+', ' ', str(x)))

In [292]:
df = df.drop_duplicates(subset=[  
    'OrderAccount', 
    'Date_booked', 
    'Date_ordered', 
    'Text_transaction',
    'Use', 
    #'Contact', 
    'AccNum', 
    'BIC', 
    'Value_transaction', 
    'Currency',
    'Information'
],keep='first').reset_index(drop=True)

In [293]:
df['SpendingType'] = np.nan

# Identify cash withdrawls
df.loc[df['Text_transaction'].isin(['BARGELDAUSZAHLUNG']),'SpendingType'] = 'CASHOUT'

# Rent
df.loc[(df['Use'].apply(lambda x: 'miete' in x.lower()))&(df['Value_transaction']<=0),'SpendingType'] = 'RENT'

# Utilities
df.loc[(df['SpendingType'].isna())&(df['Contact'].apply(lambda x: 'estw erlangerstadtwerkeag' in x.lower())),'SpendingType'] = 'UTILITIES POWER'
df.loc[(df['SpendingType'].isna())&(df['Contact'].apply(lambda x: 'm net tele' in x.lower())),'SpendingType'] = 'UTILITIES INET'

# Phone bill
df.loc[(df['Use'].apply(lambda x: 'mobilfunk' in x.lower()))&(df['Contact'].apply(lambda x: 'telekom' in x.lower()))&(df['Value_transaction']<=0),'SpendingType'] = 'UTILITIES PHONEBILL'

# Identify salary
df.loc[(df['Text_transaction'].apply(lambda x: 'lohn gehalt' in x.lower())),'SpendingType'] = 'SALARY'

# Casino spending at SHS
df.loc[df['Use'].apply(lambda x: 'siemens casino card' in x.lower()),'SpendingType'] = 'FOOD&BEVERAGES'

# Union fees
df.loc[df['Contact']=='IGM Erlangen','SpendingType'] = 'MEMBERSHIP_FEES IGMETALL'

# ImmoScout fees
df.loc[df['Use'].apply(lambda x: 'immobilien' in x.lower()),'SpendingType'] = 'MEMBERSHIP_FEES IMMOSCOUT'

# Gym fees
df.loc[df['Contact']=='SLF Sportland Franken GmbH Co KG','SpendingType'] = 'MEMBERSHIP_FEES SPORTLAND'
df.loc[df['Use'].apply(lambda x: 'kieser training' in x.lower()),'SpendingType'] = 'MEMBERSHIP_FEES KIESER'

# Bonus payments from family
df.loc[df['Use'].isin([
    'MIETZUSCHUs F STUDENT F U B DEINHARD ', 
    'MIETE UND ANDERES ',
    'SVWZ TASCHENGELD', 
    'SVWZ MIETZUSCHUs F STUDENT F U B DEINHARD',
    'SVWZ MIETE UND ANDERES'
]),'SpendingType'] = 'BONUSIN_FAMILY'
df.loc[(df['SpendingType'].isna())&(df['Use']=='TASCHENGELD ')&(df['Contact']=='Hermann Kuespert'),'SpendingType'] = 'BONUSIN_FAMILY'

# Insurance fees
df.loc[(df['Contact'].apply(lambda x: 'hansemerkur' in x.lower()))|(df['Contact']=='Techniker Krankenkasse'),'SpendingType'] = 'INSURANCE TKK'
df.loc[(df['Contact'].apply(lambda x: 'huk coburg' in x.lower()))&(df['SpendingType'].isna()),'SpendingType'] = 'INSURANCE HUK'
df.loc[(df['SpendingType'].isna())&(df['Use'].apply(lambda x: 'haftpflicht' in x.lower())),'SpendingType'] = 'INSURANCE HAFTPFL'
df.loc[(df['SpendingType'].isna())&(df['Contact']=='ARAG KRANKENVERSICHERUNG AG'),'SpendingType'] = 'INSURANCE ARAG' 
df.loc[(df['SpendingType'].isna())&(df['Contact']=='Siemens BKK'),'SpendingType'] = 'INSURANCE SBK'

# Credit card bill
df.loc[df['Contact']=='KREDITKARTENABRECHNUNG','SpendingType'] = 'CREDITCARD'

# Contact lenses
df.loc[(df['Use'].apply(lambda x: 'schaefer nohe' in x.lower()))|(df['Contact'].apply(lambda x: 'schaefer nohe' in x.lower())),'SpendingType'] = 'MEDICAL OPTICIAN'

# Gene sequencing 23andMe
df.loc[df['Use'].apply(lambda x: '23andme' in x.lower()),'SpendingType'] = 'MEDICAL MISC'

# Amazon videos
df.loc[df['Contact']=='AMAZON INSTANT VIDEO GERMANY GMBH','SpendingType'] = 'AMAZONVIDEOS'
df.loc[df['Use'].apply(lambda x: 'prime video' in x.lower()),'SpendingType'] = 'AMAZONVIDEOS'

# Amazon shopping
df.loc[(df['SpendingType'].isna())&(df['Use'].apply(lambda x: 'amzn mktp' in x.lower() or 'amazon' in x.lower())),'SpendingType'] = 'AMAZONSHOPPING'

# Entertainment
df.loc[df['Use'].apply(lambda x: 
        'spotify' in x.lower() or 
        'dancarlin' in x.lower() or
        'steam games' in x.lower()
),'SpendingType'] = 'ENTERTAINMENTMISC'

# Depot
df.loc[df['Text_transaction']=='WERTPAPIERE','SpendingType'] = 'STOCKDEPOT'

# Debit card bill
df.loc[(df['Use'].apply(lambda x: 'debitk' in x.lower()))&(df['SpendingType'].isna()),'SpendingType'] = 'DEBITCARD PAYMENT'
df.loc[(df['Text_transaction'].apply(lambda x: 'sepa' in x.lower()))&(df['SpendingType'].isna()),'SpendingType'] = 'DEBITCARD TRANSFER'

# Lent money backflow
df.loc[(df['SpendingType'].isna())&(df['Text_transaction'].apply(lambda x: 'gutschr' in x.lower())),'SpendingType'] = 'LENTBACK_PRIVATE'

# Rent deposit
df.loc[(df['SpendingType'].isna())&(df['Use'].apply(lambda x: 'kaution' in x.lower())),'SpendingType'] = 'DEPOSIT'

# Miscellaneous PayPal expenses
df.loc[(df['SpendingType'].isna())&(df['Contact'].apply(lambda x: 'paypal'in x.lower())),'SpendingType'] = 'PAYPAL_MISC'

# Costs for the account
df.loc[(df['SpendingType'].isna())&(df['AccNum'].apply(lambda x: x=='0000000000' or str(x)=='nan'))&(df['Use'].apply(lambda x: 'entgeltabrechnung' in x.lower())),'SpendingType'] = 'BANKACCOUNTCOST'
df.loc[(df['SpendingType'].isna())&(df['AccNum']=='0020557849')&(df['BIC']=='77350110')&(df['Text_transaction']=='ABSCHLUSS'),'SpendingType'] = 'BANKACCOUNTCOST'

# Label known miscellanii
known_misc = pickle.load(open(newdatapath+'05_METADATA\\known_misc.pkl','rb'))
df.loc[(df['SpendingType'].isna())&(df[['Text_transaction','Use','Contact','AccNum','BIC']].apply(lambda x: '|||'.join([str(x) for x in list(x)]) in known_misc,axis=1)),'SpendingType'] = 'KNOWNMISC'

# Identify unknown miscellanii
df.loc[df['SpendingType'].isna(),'SpendingType'] = '__UNKNOWN__' 

In [294]:
# Major expense groupings
dict_spending_type2group = {
    'CASHOUT':'CASHOUT', 
    'BANKACCOUNTCOST':'BANKACCOUNTCOST', 
    'RENT':'RENT', 
    'MEMBERSHIP_FEES SPORTLAND':'MEMBERSHIP_FEES',
    'FOOD&BEVERAGES':'UTILITIES', 
    'UTILITIES PHONEBILL':'UTILITIES', 
    'UTILITIES POWER':'UTILITIES',
    'MEMBERSHIP_FEES IGMETALL':'MEMBERSHIP_FEES', 
    'SALARY':'SALARY', 
    'AMAZONVIDEOS':'AMAZON',
    'ENTERTAINMENTMISC':'MISC', 
    'CREDITCARD':'CREDITCARDBILL', 
    'MEDICAL OPTICIAN':'MEDICAL',
    'UTILITIES INET':'UTILITIES', 
    'MEMBERSHIP_FEES KIESER':'MEMBERSHIP_FEES', 
    'DEBITCARD PAYMENT':'CREDITCARDBILL',
    'PAYPAL_MISC':'MISC', 
    'DEBITCARD TRANSFER':'MISC',
    'LENTBACK_PRIVATE':'LENTBACK',
    'INSURANCE SBK':'INSURANCE', 
    'BONUSIN_FAMILY':'BONUS', 
    'STOCKDEPOT':'STOCKDEPOT', 
    'AMAZONSHOPPING':'AMAZON',
    'MEMBERSHIP_FEES IMMOSCOUT':'MEMBERSHIP_FEES', 
    'INSURANCE ARAG':'INSURANCE', 
    'KNOWNMISC':'KNOWNMISC',
    'INSURANCE HAFTPFL':'INSURANCE', 
    'INSURANCE TKK':'INSURANCE', 
    'MEDICAL MISC':'MEDICAL',
    'INSURANCE HUK':'INSURANCE', 
    'DEPOSIT':'DEPOSIT',
    '__UNKNOWN__':'__UNKNOWN__'
}
df['ExpenseGroup'] = df['SpendingType'].map(dict_spending_type2group)

In [295]:
df.to_excel(storepath+'STF.xlsx')

In [296]:
# Extract relevant information
tf = df[['Date_booked','Value_transaction']]
tf = tf.drop(index=tf[tf['Value_transaction']>=0].index).reset_index(drop=True)

In [297]:
# Generate interesting time indices
tf['YYYY-MM'] = pd.to_datetime(tf['Date_booked'].apply(lambda x: x.strftime('%Y-%m')))
tf['YYYY-MM-DD'] = pd.to_datetime(tf['Date_booked'].apply(lambda x: x.strftime('%Y-%m-%d')))
# Extract single date identifiers
#tf['DD'] = tf['Date_booked'].apply(lambda x: x.day)
#tf['MM'] = tf['Date_booked'].apply(lambda x: x.month)
#tf['YYYY'] = tf['Date_booked'].apply(lambda x: x.year)
# Drop unformatted column
tf = tf.drop(columns=['Date_booked'])

In [298]:
# Accumulate by months and days 
rf_monthly = tf[['YYYY-MM','Value_transaction']].groupby(['YYYY-MM']).sum().abs()
rf_daily = tf[['YYYY-MM-DD','Value_transaction']].groupby(['YYYY-MM-DD']).sum().abs()

In [299]:
# Extract full date
rf_monthly['Date'] = rf_monthly.apply(lambda x: x.index)
rf_daily['Date'] = rf_daily.apply(lambda x: x.index)
# Extract years
rf_daily['Year'] = rf_daily.apply(lambda x: x['Date'].year,axis=1)
rf_monthly['Year'] = rf_monthly.apply(lambda x: x['Date'].year,axis=1)
# Extract months
rf_daily['Month'] = rf_daily.apply(lambda x: x['Date'].month,axis=1)
rf_monthly['Month'] = rf_monthly.apply(lambda x: x['Date'].month,axis=1)
# Extract days
rf_daily['Day'] = rf_daily.apply(lambda x: x['Date'].day,axis=1)
rf_monthly['Day'] = rf_monthly.apply(lambda x: x['Date'].day,axis=1)

In [300]:
# Determine current dates of interest
c_date = tf['YYYY-MM-DD'].max()
p_date = c_date-pd.DateOffset(months=1)
pp_date = c_date-pd.DateOffset(months=2)
ppp_date = c_date-pd.DateOffset(months=3)

In [301]:
# Current month
start_date = '{}-{}-{}'.format(str(c_date.year),str(str(c_date.month).zfill(2)),'01')
end_date = '{}-{}-{}'.format(str(c_date.year if c_date.month!=12 else c_date.year+1),str(str((c_date.month)%12+1).zfill(2)),'01')
c_index = pd.date_range(
    start=start_date,
    end=end_date
)[:-1]
tt = rf_daily[(rf_daily['Year']==c_date.year)&(rf_daily['Month']==c_date.month)][['Value_transaction']]
m0 = tt.reindex(c_index,fill_value=0)

In [302]:
# Current month - 1
start_date = '{}-{}-{}'.format(str(p_date.year),str(str(p_date.month).zfill(2)),'01')
end_date = '{}-{}-{}'.format(str(p_date.year if p_date.month!=12 else p_date.year+1),str(str((p_date.month)%12+1).zfill(2)),'01')
c_index = pd.date_range(
    start=start_date,
    end=end_date
)[:-1]
tt = rf_daily[(rf_daily['Year']==p_date.year)&(rf_daily['Month']==p_date.month)][['Value_transaction']]
m1 = tt.reindex(c_index,fill_value=0)

In [303]:
# Current month - 2
start_date = '{}-{}-{}'.format(str(pp_date.year),str(str(pp_date.month).zfill(2)),'01')
end_date = '{}-{}-{}'.format(str(pp_date.year if pp_date.month!=12 else pp_date.year+1),str(str((pp_date.month)%12+1).zfill(2)),'01')
c_index = pd.date_range(
    start=start_date,
    end=end_date
)[:-1]
tt = rf_daily[(rf_daily['Year']==pp_date.year)&(rf_daily['Month']==pp_date.month)][['Value_transaction']]
m2 = tt.reindex(c_index,fill_value=0)

In [304]:
# Current month - 3
start_date = '{}-{}-{}'.format(str(ppp_date.year),str(str(ppp_date.month).zfill(2)),'01')
end_date = '{}-{}-{}'.format(str(ppp_date.year if ppp_date.month!=12 else ppp_date.year+1),str(str((ppp_date.month)%12+1).zfill(2)),'01')
c_index = pd.date_range(
    start=start_date,
    end=end_date
)[:-1]
tt = rf_daily[(rf_daily['Year']==ppp_date.year)&(rf_daily['Month']==ppp_date.month)][['Value_transaction']]
m3 = tt.reindex(c_index,fill_value=0)

In [305]:
writer = pd.ExcelWriter(storepath+'RunningMonthlyExp.xlsx',engine='openpyxl')

m0.to_excel(writer,sheet_name='p0')
m1.to_excel(writer,sheet_name='p1')
m2.to_excel(writer,sheet_name='p2')
m3.to_excel(writer,sheet_name='p3')

In [306]:
bf = df[['Date_booked','Value_transaction']]

pf = bf[bf['Value_transaction']>0].reset_index(drop=True)
nf = bf[bf['Value_transaction']<0].reset_index(drop=True)

pf['YYYY-MM'] = pd.to_datetime(pf['Date_booked'].apply(lambda x: x.strftime('%m-%Y')))
nf['YYYY-MM'] = pd.to_datetime(nf['Date_booked'].apply(lambda x: x.strftime('%m-%Y')))

In [307]:
pf = pf.drop(columns=['Date_booked'])
nf = nf.drop(columns=['Date_booked'])

In [308]:
pf = pf.groupby(['YYYY-MM']).sum()
nf = nf.groupby(['YYYY-MM']).sum()

In [309]:
mf = pd.merge(nf,pf,on='YYYY-MM').rename(columns={
    'Value_transaction_x':'expenses',
    'Value_transaction_y':'revenue'
})
mf['expenses'] = mf['expenses'].abs()

In [310]:
mf.to_excel(storepath+'MonthlyExpRevAggregate.xlsx')

In [311]:
at = pd.read_excel(newdatapath+'02_AccountTotal\\AccountTracker.xlsx',parse_dates=['Date'],index_col='Date')

In [312]:
at.to_excel(storepath+'ATR.xlsx')